In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    #rotation_range=40,
    #width_shift_range=0.2,
    #height_shift_range=0.2,
    #shear_range=0.2,
    #zoom_range=0.2,
    #channel_shift_range=10,
    #horizontal_flip=True,
    #vertical_flip=True,
    #fill_mode='nearest',
    validation_split=0.2,
    rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    directory='/content/drive/MyDrive/AI_共用/AI_辨識食物類別/tw_food_101',
    target_size=(200,200),
    batch_size=16,
    subset='training')

validation_generator = train_datagen.flow_from_directory(
    directory='/content/drive/MyDrive/AI_共用/AI_辨識食物類別/tw_food_101',
    target_size=(200,200),
    batch_size=16,
    subset='validation')

Found 16322 images belonging to 101 classes.
Found 4049 images belonging to 101 classes.


In [12]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img

from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping

resnet_model = ResNet50(weights="imagenet", include_top=False, input_shape=(200, 200, 3))

In [13]:
#X_train = preprocess_input(train_generator)

In [14]:
model = Sequential()
model.add(resnet_model)
model.add(Dropout(0.5))
model.add(GlobalAveragePooling2D())
#model.add(Dense(50))
model.add(Dropout(0.5))
model.add(Dense(101, activation="softmax"))
model.summary()   # 顯示模型摘要資訊

resnet_model.trainable = False

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 dropout_2 (Dropout)         (None, 7, 7, 2048)        0         
                                                                 
 global_average_pooling2d_1   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dropout_3 (Dropout)         (None, 2048)              0         
                                                                 
 dense_1 (Dense)             (None, 101)               206949    
                                                                 
Total params: 23,794,661
Trainable params: 23,741,541
Non-trainable params: 53,120
_____________________________________

In [15]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [16]:
es = EarlyStopping(monitor = "accuracy", mode = "auto", patience = 3, verbose = 1)
mc = ModelCheckpoint("/content/drive/MyDrive/AI_共用/AI_辨識食物類別/best_model.h5", monitor="accuracy", mode="auto", verbose=1,
                     save_best_only=True)

In [17]:
import time
time_start = time.time() #開始計時

In [18]:
history = model.fit_generator(
    train_generator,
    epochs=50,
    validation_data = validation_generator,
    callbacks = [es, mc])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """


Epoch 1/50
  34/1021 [..............................] - ETA: 1:30:01 - loss: 5.3300 - accuracy: 0.0037

/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 3004 bytes but only got 0. Skipping tag 37500
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 2. Skipping tag 41486
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 41487
  " Skipping tag %s" % (size, len(data), tag)


 137/1021 [===>..........................] - ETA: 1:20:19 - loss: 5.0903 - accuracy: 0.0137

/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 256 bytes but only got 230. Skipping tag 50341
  " Skipping tag %s" % (size, len(data), tag)


 748/1021 [====================>.........] - ETA: 25:31 - loss: 4.9110 - accuracy: 0.0115

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


1021/1021 [==============================] - ETA: 0s - loss: 4.8761 - accuracy: 0.0126
Epoch 1: accuracy improved from -inf to 0.01262, saving model to /content/drive/MyDrive/AI_共用/AI_辨識食物類別/best_model.h5
1021/1021 [==============================] - 7280s 7s/step - loss: 4.8761 - accuracy: 0.0126 - val_loss: 4.6381 - val_accuracy: 0.0203
Epoch 2/50
1021/1021 [==============================] - ETA: 0s - loss: 4.7133 - accuracy: 0.0206
Epoch 2: accuracy improved from 0.01262 to 0.02059, saving model to /content/drive/MyDrive/AI_共用/AI_辨識食物類別/best_model.h5
1021/1021 [==============================] - 341s 334ms/step - loss: 4.7133 - accuracy: 0.0206 - val_loss: 4.5699 - val_accuracy: 0.0232
Epoch 3/50
1021/1021 [==============================] - ETA: 0s - loss: 4.6734 - accuracy: 0.0204
Epoch 3: accuracy did not improve from 0.02059
1021/1021 [==============================] - 326s 319ms/step - loss: 4.6734 - accuracy: 0.0204 - val_loss: 4.5577 - val_accuracy: 0.0301
Epoch 4/50
1021/1021 [

KeyboardInterrupt: ignored

In [ ]:
time_end = time.time()    #結束計時

time_c= time_end - time_start   #執行所花時間
print('time cost', time_c, 's')